In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.stats as stats

In [2]:
sym_fea = pd.read_csv("../../../../extract_pubmed/2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# DMN。
# umls id---> umls name --->mesh name

ori_df = pd.read_csv("DMN_2013AA.txt",sep='|',names=["disA","disB","cos_sim"])
print(ori_df.shape)

ori_df.head()

(373527, 3)


,disA,disB,cos_sim
0,C0028326,C2673914,0.012170
1,C0342788,C2678474,0.054375
2,C0546952,C1855923,0.005606
3,C1956125,C2750069,0.003773
4,C1843366,C3151959,0.013714


In [5]:
# UMLS-Mesh
mapping_df = pd.read_csv("../DisGeNET-disease_mappings.tsv",sep="\t")
mapping_df = mapping_df[mapping_df["vocabulary"]=="MSH"]
mapping_df.reset_index(drop=True,inplace=True)
mapping_df[:2]

,diseaseId,name,vocabulary,code,vocabularyName
0,C1970109,AROMATASE EXCESS SYNDROME,MSH,C000591739,Aromatase Excess Syndrome
1,C1970109,AROMATASE EXCESS SYNDROME,MSH,C000591739,"Familial gynecomastia, due to increased aromat..."


### trans disease name
- 2305 diseases in DMN

#### UMLS->MSH name

In [4]:
#0、umls id->umls name
umlsToName = pd.read_csv("DMN_dname.txt",sep="\t",names=["umls","name"])
print(umlsToName.shape)
umlsToName[:2]

(2421, 2)


,umls,name
0,C1968602,"surfactant metabolism dysfunction, pulmonary, 1"
1,C0268293,corticosterone methyl oxidase type i deficiency


In [7]:
umls_trans = {}
for ite in range(len(umlsToName)):
    umls_trans[umlsToName.loc[ite,"umls"]]=umlsToName.loc[ite,"name"]

ori_df_name = ori_df.replace({"disA":umls_trans,"disB":umls_trans})
print(ori_df_name.shape)
ori_df_name[:2]

(373527, 3)


,disA,disB,cos_sim
0,noonan syndrome,"anemia, sideroblastic, pyridoxine-refractory, ...",0.012170
1,renal carnitine transport defect,"cardiomyopathy, dilated, 2a (disorder)",0.054375


In [8]:
# 1、Get unique disease in comorbidity data
DMN_dis = pd.concat([pd.DataFrame({"dis":ori_df_name["disA"]}),pd.DataFrame({"dis":ori_df_name["disB"]})])
print(len(DMN_dis)) 

DMN_dis.drop_duplicates(inplace=True)
print(len(DMN_dis))

DMN_dis.reset_index(drop=True,inplace=True)
DMN_dis[:2]

747054
2305


,dis
0,noonan syndrome
1,renal carnitine transport defect


In [11]:
# 2、->Mesh name
tar_dis_ls = sym_fea.index
tar_dis_df = pd.DataFrame(tar_dis_ls,columns=["dis"])
print(tar_dis_df.shape)
print(tar_dis_df[:5])

mapping_df["name"] = mapping_df["name"].str.lower()
join_df = pd.merge(DMN_dis,mapping_df,how='inner',left_on="dis",right_on="name")
print(join_df.shape)
join_df.head()

# join_df.to_csv("join_df.csv")

(4268, 1)
                                dis
0                Nevus, Intradermal
1                    Superinfection
2                          Coloboma
3  Abnormalities, Radiation-Induced
4         Bronchial Hyperreactivity
(2949, 6)


,dis,diseaseId,name,vocabulary,code,vocabularyName
0,noonan syndrome,C0028326,noonan syndrome,MSH,D009634,Noonan Syndrome
1,noonan syndrome,C0028326,noonan syndrome,MSH,D009634,"Turner Syndrome, Male"
2,noonan syndrome,C0028326,noonan syndrome,MSH,D009634,Female Pseudo-Turner Syndrome
3,noonan syndrome,C0028326,noonan syndrome,MSH,D009634,Noonan Syndrome 1
4,renal carnitine transport defect,C0342788,renal carnitine transport defect,MSH,C536778,Systemic carnitine deficiency


In [12]:
# 3、UMLS name --->Mesh name
trans = {}

for i in tqdm(range(len(join_df))):
    if join_df.loc[i,"vocabularyName"] in tar_dis_ls:
        UMLS_name = join_df.loc[i,"dis"]
        mesh_name = join_df.loc[i,"vocabularyName"]
        
        trans[UMLS_name]= mesh_name   
            
print(len(trans)) #412 diseases

100%|███████████████████████████████████████████████████████████████████████████| 2949/2949 [00:00<00:00, 59605.63it/s]

412


In [13]:
trans_df = pd.DataFrame.from_dict(trans,orient="index")
trans_df.reset_index(inplace=True)
trans_df.columns=["UMLS name","MSH name"]
trans_df[:2]

,UMLS name,MSH name
0,noonan syndrome,Noonan Syndrome
1,alagille syndrome 1,Alagille Syndrome


####  Extract subgraphs of DMN

In [38]:
DMN_mesh_df = pd.DataFrame()

for index in tqdm(range(len(ori_df_name))):
    disA = ori_df_name.loc[index,"disA"]
    disB = ori_df_name.loc[index,"disB"]
    cos_sim = ori_df_name.loc[index,"cos_sim"]
    
    if (disA in list(trans_df["UMLS name"])) and (disB in list(trans_df["UMLS name"])):
        DMN_mesh_df = DMN_mesh_df.append(pd.Series([trans[disA],trans[disB],cos_sim]),ignore_index=True)
        
print(DMN_mesh_df.shape) # 
DMN_mesh_df.columns = ["disA","disB","cos_sim"]
DMN_mesh_df[:2]

100%|████████████████████████████████████████████████████████████████████████| 373527/373527 [01:10<00:00, 5290.31it/s]

(15888, 3)


,disA,disB,cos_sim
0,"Spastic Paraplegia, Hereditary",Mucolipidoses,0.009291
1,Mucopolysaccharidosis III,Holocarboxylase Synthetase Deficiency,0.004953


In [39]:
# DMN_mesh_df.to_csv("DMN_2013AA_name.csv")

### Extract the corresponding model prediction results

In [40]:
predict_score = pd.read_csv("../../visualization/model_predictAll_result.csv",index_col=0)

dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in tqdm(trans_df["MSH name"]):
    index_inSym = sym_fea_index_lower.index(dis.lower())
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:5]

E:\Anaconda\envs\tf2\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
100%|█████████████████████████████████████████████████████████████████████████████| 412/412 [00:00<00:00, 26365.18it/s]

412


[2797, 4094, 480, 1632, 1576]

In [48]:
scoreABs = []
scoreBAs = []
DMN_confs = [] #DMN，（0,1）

for disA in tqdm(trans_df["MSH name"]):
    disA_index = dis_index_inSym[list(trans_df["MSH name"]).index(disA)]
    
    for disB in trans_df["MSH name"]:
        if disA==disB:
            continue
        disB_index = dis_index_inSym[list(trans_df["MSH name"]).index(disB)]
        
        con_df = DMN_mesh_df.loc[(DMN_mesh_df["disA"]==disA) & (DMN_mesh_df["disB"]==disB)]
        
        if len(con_df)>0:
            scoreAB = predict_score.loc[disA_index*4268+disB_index]
            scoreBA = predict_score.loc[disB_index*4268+disA_index]
            scoreABs.append(scoreAB)
            scoreBAs.append(scoreBA)
            
            sim_score = list(con_df["cos_sim"])[0]
            DMN_confs.append(sim_score)
        else:
            continue
            
print(len(DMN_confs),len(DMN_confs)==len(scoreABs))
DMN_confs[:2]

100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [06:35<00:00,  1.04it/s]


In [49]:
print(len(DMN_confs),len(DMN_confs)==len(scoreABs))
DMN_confs[:2]

22694 True


[0.0332050698636417, 0.0126422579174373]

In [50]:

scores_predict=[] 
scores_predict = np.sum([scoreABs,scoreBAs],axis=0)/2
scores_predict = scores_predict.squeeze()

print(scores_predict[:10])

[0.04102472 0.00495054 0.27993447 0.03995889 0.30943926 0.00290634
 0.18209699 0.00539309 0.11116414 0.03785008]


In [51]:
DMN_sim = []
DMN_unsim = []

for i in range(len(scores_predict)):
    score = scores_predict[i]
    if score>=0.5:
        DMN_sim.append(DMN_confs[i])
    elif score<0.5:
        DMN_unsim.append(DMN_confs[i])
        
print(len(DMN_sim),"--",len(DMN_unsim))

4737 -- 17957


#### Calculate distribution difference

In [52]:
distribution_dif = stats.mannwhitneyu(DMN_sim,DMN_unsim,alternative='two-sided')
distribution_dif

MannwhitneyuResult(statistic=49627313.0, pvalue=4.8214053046716285e-70)

In [53]:
print(np.mean(DMN_sim))
print(np.mean(DMN_unsim))

0.020411612827446156
0.015450777880477813


'相似组的DMN边权较大，√'

In [66]:
DMN_sim.sort(reverse=True)
DMN_sim[:25]

[0.463100239853398,
 0.4561010231369063,
 0.4064609973841454,
 0.3910494245072792,
 0.3507983542338357,
 0.3221971379575166,
 0.3221971379575166,
 0.3221971379575166,
 0.3058117353700386,
 0.2812802966906937,
 0.2812802966906937,
 0.2812802966906937,
 0.2693272547847553,
 0.261482117883078,
 0.2298161697347929,
 0.1983246526674052,
 0.1912328445758419,
 0.1826351643258574,
 0.1722587359695794,
 0.1722587359695794,
 0.1722587359695794,
 0.1555282354100481,
 0.1523364192290587,
 0.150890396264842,
 0.150890396264842]

In [62]:
DMN_unsim.sort(reverse=True)
DMN_unsim[:100]

[0.5918048194314739,
 0.4601767155037639,
 0.4601767155037639,
 0.3614142445187476,
 0.3614142445187476,
 0.2892404044598641,
 0.2701270773860166,
 0.2701270773860166,
 0.2481589609430238,
 0.2135025405487935,
 0.2034706891890647,
 0.1977920563558424,
 0.1924812239829524,
 0.1812183287118736,
 0.1812183287118736,
 0.1727980314039409,
 0.1678158996718022,
 0.1678158996718022,
 0.1660177357852365,
 0.1660177357852365,
 0.1496250321560855,
 0.1486510486737694,
 0.148333614002908,
 0.148333614002908,
 0.144689372897683,
 0.1442685242817061,
 0.1436327798920529,
 0.1360766201970596,
 0.1360766201970596,
 0.1360766201970596,
 0.1357098603361308,
 0.1345918551619211,
 0.1284221900131329,
 0.127938078018569,
 0.127938078018569,
 0.127938078018569,
 0.1240328412172383,
 0.1226169171236531,
 0.121352794518979,
 0.1206083517361042,
 0.1206083517361042,
 0.1201449467742209,
 0.1180950516677462,
 0.1180911176268262,
 0.1159486807748876,
 0.1159486807748876,
 0.1159486807748876,
 0.1158048525800453,

In [64]:
# pd.Series(DMN_sim).to_csv("DMN_sim.csv",index="cos_sim")
# pd.Series(DMN_unsim).to_csv("DMN_unsim.csv",index="cos_sim")